In [6]:
# Fetch hourly weather data for multiple hours/days

from urllib import request, parse
import json, csv, os

WX_URL = "https://api.open-meteo.com/v1/forecast"

# ---- settings ----
lat, lon = 39.73915, -104.9847   # Denver coords
out_csv = "weather_hourly.csv"
days = 3  # how many days ahead
# ------------------

def fetch_hourly(lat, lon, days=1):
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,windspeed_10m,winddirection_10m,weathercode",
        "forecast_days": days,
        "timezone": "auto",
    }
    url = f"{WX_URL}?{parse.urlencode(params)}"
    with request.urlopen(url, timeout=30) as resp:
        return json.loads(resp.read().decode("utf-8"))

data = fetch_hourly(lat, lon, days)

hours = data["hourly"]
rows = []
for i, t in enumerate(hours["time"]):
    rows.append({
        "timestamp": t,
        "latitude": data["latitude"],
        "longitude": data["longitude"],
        "timezone": data["timezone"],
        "temperature_c": hours["temperature_2m"][i],
        "windspeed_kmh": hours["windspeed_10m"][i],
        "winddirection_deg": hours["winddirection_10m"][i],
        "weathercode": hours["weathercode"][i],
    })

# Save to CSV
cols = list(rows[0].keys())
with open(out_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=cols)
    w.writeheader()
    w.writerows(rows)

print(f"[done] Wrote {len(rows)} rows to {os.path.abspath(out_csv)}")
print("First 3 rows:", rows[:3])


[done] Wrote 72 rows to /home/x/Workspace/datascience/weather_hourly.csv
First 3 rows: [{'timestamp': '2025-09-01T00:00', 'latitude': 39.746895, 'longitude': -104.987076, 'timezone': 'America/Denver', 'temperature_c': 15.1, 'windspeed_kmh': 7.6, 'winddirection_deg': 265, 'weathercode': 0}, {'timestamp': '2025-09-01T01:00', 'latitude': 39.746895, 'longitude': -104.987076, 'timezone': 'America/Denver', 'temperature_c': 13.9, 'windspeed_kmh': 1.0, 'winddirection_deg': 135, 'weathercode': 0}, {'timestamp': '2025-09-01T02:00', 'latitude': 39.746895, 'longitude': -104.987076, 'timezone': 'America/Denver', 'temperature_c': 13.3, 'windspeed_kmh': 5.9, 'winddirection_deg': 166, 'weathercode': 0}]
